In [1]:
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
df = pd.read_csv('/content/train.csv')

In [3]:
df = df.sort_values(by=['id_seqpos'], key=lambda x: x.str.extract(r'_(\d+)$').astype(int).iloc[:, 0])

In [4]:
a = ['b1_structure','b2_structure','b3_structure','b4_structure','b5_structure']
for i in a:
   df[i] = df[i].replace({'.':2 , '(': 0 , ')':1,'-1':-1})
b = ['b1_sequence','b2_sequence','b3_sequence','b4_sequence','b5_sequence']
for e in b:
   df[e] = df[e].replace({'A':3 , 'C': 4 , 'G':5,'U':6,'-1':-1})
c = ['b1_predicted_loop_type','b2_predicted_loop_type','b3_predicted_loop_type','b4_predicted_loop_type','b5_predicted_loop_type']
for t in c:
   df[t] = df[t].replace({'B':7 , 'E': 8 , 'H':9,'I':10,'M':11,'S':12,'X':13,'-1':-1})
d = ['sequence','a1_sequence','a2_sequence','a3_sequence','a4_sequence','a5_sequence']
for q in d:   
  df[q] = df[q].replace({'A':3 , 'C': 4 , 'G':5,'U':6})
e = ['structure','a1_structure','a2_structure','a3_structure','a4_structure','a5_structure']  
for p in e:
  df[p] = df[p].replace({'.':2 , '(': 0 , ')':1})
f = ['predicted_loop_type','a1_predicted_loop_type','a1_predicted_loop_type','a2_predicted_loop_type','a3_predicted_loop_type','a4_predicted_loop_type','a5_predicted_loop_type']  
for o in f:
  df[o] = df[o].replace({'B':7 , 'E': 8 , 'H':9,'I':10,'M':11,'S':12,'X':13})

In [5]:
X = df.drop(['id','id_seqpos','reactivity','deg_Mg_pH10','deg_Mg_50C'],axis=1)
y = df[['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']]


In [6]:
from sklearn.model_selection import train_test_split , KFold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, random_state=40)

In [7]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(max_depth=6,learning_rate=0.18,colsample_bytree=0.9,reg_alpha=5,reg_lambda=6
                      ,min_child_weight = 2,n_estimators=90000)


xgb_model.fit(X_train, y_train, early_stopping_rounds=8,eval_set=[(X_test, y_test)],eval_metric='rmse', verbose=False)

y_pred_xgb = xgb_model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [8]:
def MCRMSE_numpy(y_true, y_pred):
    colwise_mse = np.mean(np.square(y_true - y_pred), axis=1)
    return np.mean(np.sqrt(colwise_mse))

In [9]:
print(MCRMSE_numpy(y_test,y_pred_xgb))
print(xgb_model.score(X_train,y_train) * 100)
print(xgb_model.score(X_test,y_test) * 100)

0.17275479409248995
87.53002245868574
62.40999848781593


In [ ]:
from sklearn.metrics import make_scorer
def mcrmse (y_true,y_pred,N=3):
  assert len(y_true) == len(y_pred)
  n = len(y_true)
  return np.sum(np.sqrt(np.sum((y_true - y_pred)**2,axis=0)/n)) / N
aa = make_scorer(mcrmse , greater_is_better = False)
print(mcrmse(xgb_model.predict(X_test) , np.array(y_test)))
print(mcrmse(xgb_model.predict(X_train) , np.array(y_train)))

0.3507883082460285
0.2094439373124408


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
a = ['b1_structure','b2_structure','b3_structure','b4_structure','b5_structure']
for i in a:
   test[i] = test[i].replace({'.':0 , '(': 1 , ')':2,'-1':3})
b = ['b1_sequence','b2_sequence','b3_sequence','b4_sequence','b5_sequence']
for e in b:
   test[e] = test[e].replace({'A':0 , 'C': 1 , 'G':2,'U':3,'-1':4})
c = ['b1_predicted_loop_type','b2_predicted_loop_type','b3_predicted_loop_type','b4_predicted_loop_type','b5_predicted_loop_type']
for t in c:
   test[t] = test[t].replace({'B':0 , 'E': 1 , 'H':2,'I':3,'M':4,'S':5,'X':6,'-1':7})
d = ['sequence','a1_sequence','a2_sequence','a3_sequence','a4_sequence','a5_sequence']
for q in d:   
  test[q] = test[q].replace({'A':0 , 'C': 1 , 'G':2,'U':3})
e = ['structure','a1_structure','a2_structure','a3_structure','a4_structure','a5_structure']  
for p in e:
  test[p] = test[p].replace({'.':0 , '(': 1 , ')':2})
f = ['predicted_loop_type','a1_predicted_loop_type','a1_predicted_loop_type','a2_predicted_loop_type','a3_predicted_loop_type','a4_predicted_loop_type','a5_predicted_loop_type']  
for o in f:
  test[o] = test[o].replace({'B':0 , 'E': 1 , 'H':2,'I':3,'M':4,'S':5,'X':6})

In [ ]:
ID = test['id_seqpos']
TEST = test.drop(['id_seqpos','id'],axis=1)
a = xgb_model.predict(TEST)
new = pd.DataFrame(a,columns=['reactivity','deg_Mg_pH10','deg_Mg_50C'])

In [ ]:
new_data = pd.concat([ID.rename('id_seqpos'),new], axis=1)

In [ ]:
new_data.to_csv('xgboost_new_tunning.csv',index=False)

In [ ]:
new_data

,id_seqpos,reactivity,deg_Mg_pH10,deg_Mg_50C
0,id_001f94081_0,0.651161,1.303060,0.778812
1,id_001f94081_4,0.879710,0.726091,0.711568
2,id_001f94081_8,0.140349,0.204849,0.103310
3,id_001f94081_9,0.524826,0.649451,0.965863
4,id_001f94081_13,0.893850,0.886185,1.040395
...,...,...,...,...
32635,id_fff546103_39,0.007572,0.165827,0.045841
32636,id_fff546103_52,0.061107,0.160196,0.061730
32637,id_fff546103_62,0.399856,0.350042,0.358902
32638,id_fff546103_65,0.093712,0.218223,0.183091
